<a href="https://colab.research.google.com/github/nlpbokproject/NLP_BOK_Team/blob/master/NCB_step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from collections import defaultdict
import pandas as pd 
import numpy as np
import math
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
k =0.5

input_file = pd.read_csv("./DATA/NBC/NBC-merged.csv").dropna()
input_file['date']=input_file['date']
#input_file = pd.read_csv("./DATA/NBC/NBC-yeonhap_info_step1-1.csv")
training_set = np.array(input_file[['ngram+unigram','label']])
#print(training_set)

In [0]:
df = input_file[input_file.label=='1']
df[['date']]
df.shape

In [0]:

# 범주에 속하는 토큰수 세기 1(예. 긍정), 0(예. 부정))
doccnt1 = 0
doccnt0 = 0

#input_file = pd.read_csv(file,lineterminator='\n')
    
# 토큰별로 문서내 빈도수 카운팅
wordfreq = defaultdict(lambda : [0, 0])
for doc, point in training_set:
    words = doc.replace('@@@',',').split('.')
    
    for word in words :
        
        #my_df['words']=word
        if point == '1' :
            wordfreq[word][1] += 1
            #print(word+" 1")
        elif point == '-1' :
            #print(word + " : "  + point)
            wordfreq[word][0] += 1
            #wordfreq[word][0] += 1
        else:
            print("nothing : "+word+" "+point)
            
for key, (cnt0, cnt1) in wordfreq.items() :
    if cnt1 > 0 :
        doccnt1 += cnt1
    if cnt0 > 0 :
        doccnt0 += cnt0
        
        
pd.DataFrame.from_dict(wordfreq, orient='index').to_csv('./DATA/NBC/wordfreq.csv')
#print(wordfreq)

In [0]:
test = pd.read_csv('./DATA/NBC/wordfreq.csv')


In [0]:

# 범주에 속하는 토큰수 세기 1(예. 긍정), 0(예. 부정))
doccnt1 = 0
doccnt0 = 0

#input_file = pd.read_csv(file,lineterminator='\n')
    
# 토큰별로 문서내 빈도수 카운팅
wordfreq = defaultdict(lambda : [0, 0])
for doc, point in training_set:
    words = doc.replace('@@@',',').split('.')
    
    for word in words :
        
        #my_df['words']=word
        if point == '1' :
            wordfreq[word][1] += 1
            #print(word+" 1")
        elif point == '-1' :
            #print(word + " : "  + point)
            wordfreq[word][0] += 1
            #wordfreq[word][0] += 1
        else:
            print("nothing : "+word+" "+point)
            
for key, (cnt0, cnt1) in wordfreq.items() :
    if cnt1 > 0 :
        doccnt1 += cnt1
    if cnt0 > 0 :
        doccnt0 += cnt0
        
        
pd.DataFrame.from_dict(wordfreq, orient='index').to_csv('./DATA/NBC/wordfreq.csv')
#print(wordfreq)

In [0]:
#190527 새로운 NBC-merged에 대해 수정
from nltk.classify import NaiveBayesClassifier
# training : 토큰별 조건부 확률 계산
intensity_cutoff=1.3
intensity_cutoff2=0.76
eps = 1e-6


scores = []
wordprobs = defaultdict(lambda : [0, 0])
for key, (cnt0, cnt1) in wordfreq.items():
    wordprobs[key][0] = (cnt0 + k) / (doccnt0 + 2*k)
    wordprobs[key][1] = (cnt1 + k) / (doccnt1 + 2*k)
    
    pos_score = wordprobs[key][1]
    neg_score = wordprobs[key][0]
    
    polarity = pos_score - neg_score
    polarity_score = pos_score/neg_score
    

    intensity = pos_score / (neg_score + eps) if polarity > 0 else neg_score / (pos_score + eps)
    polarity_score = polarity_score if (intensity > intensity_cutoff) else 0
    
    label = 1 if polarity_score > 1 else '0'
    label = -1 if polarity_score == 0 else label
    #test =  {'key':key, 'polarity':polarity,'Polarity score': polarity_score, 'Intensity': intensity}
    score = (key, cnt1, cnt0, wordprobs[key][1], wordprobs[key][0], polarity,polarity_score,intensity,label)
    scores.append(score)
    

# print(wordprobs)

# word,label,polarity,ratio,intensity

dic = pd.DataFrame(scores, columns=['ngrams', 'counting_h', 'counting_d', 'p_h', 'p_d','polarity','Polarity score', 'Intensity','label'])

#pd.merge(dic,file1,on='ngrams')

In [0]:
k = 0.5

intensity_cutoff=1.3
intensity_cutoff2=0.76

dic["w|pos"] = (dic['counting_h']+k)/(dic['counting_h'].sum()+k*2)
dic["w|neg"] = (dic['counting_d']+k)/(dic['counting_d'].sum()+k*2)
dic["polarity score"] = dic["w|pos"]/dic["w|neg"]
dic["intensity"] = [x if x > 1 else 1/x for x in dic['polarity score']]

#dic["intensity"] = [x if (x < intensity_cutoff)&(x > intensity_cutoff2) else 0] 

#dic["polarity score"] = dic["polarity score"] if (dic["intensity"] > intensity_cutoff)&(dic["intensity"] < intensity_cutoff2) else 0
dic["lable"] = [-1 if x < 1 else 1 for x in dic['polarity score']] # lable가-1이면 비둘기파. 1이면 매파.

"""dic["lable"] = [1 if x > 1 else -1 for x in dic["polarity score"]]
dic["lable"] = [0 if x == 0 else label for x in dic["polarity score"]]"""
#dic["lable"] = [1 if polarity_score > 1 else -1]
#dic["lable"] = [0 if polarity_score == 0 else label]


score = (key, cnt1, cnt0, wordprobs[key][1], wordprobs[key][0], polarity,polarity_score,intensity,label)
scores.append(score)
#dic = pd.DataFrame(scores, columns=['ngrams', 'counting_h', 'counting_d', 'p_h', 'p_d','polarity','Polarity score', 'Intensity','label'])

In [0]:
dic_final = dic[dic.intensity > 1.3][["ngrams", "counting_h", "counting_d", "w|pos", "w|neg", "polarity score", "intensity", "lable"]]
#dic_final.to_csv("./DATA/NBC/NBC_step2.csv")
#final

dic_final.rename(['lable':'label'])

dic_final